In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class Node:
  def __init__(self):
    self.child = [None, None]
  
  def __setitem__(self, k: bool, _):
    if self.child[k] is None: self.child[k] = Node() 
  
  def __getitem__(self, k: bool):
    if self.child[k]: return self.child[k]
    self.child[k] = Node()
    return self.child[k]
  
  def __contains__(self, k: bool):
    return self.child[k] is not None

class BinaryTrie:
  def __init__(self, depth=32):
    self.root = Node()
    self.depth = depth

  def __contains__(self, n) :
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False
      cur = cur[v]
    return True
  
  def __repr__(self): #print 모든 leaf노드의 값 출력. O(2^N) 이지만 보통 sparse하므로 보통 O(|leaf nodes|)
    cur = self.root
    if all(cur.child) : return "[]"
    S = [(cur, 0, 0)]
    res = []
    while S :
      u, d, v = S.pop()
      if not u.child[0] and not u.child[1] and d == self.depth:
        res.append(str(v))
        continue
        
      for i in range(2) :
        if u.child[i] :
          S.append((u.child[i], d+1, v*2+i))
      
    return f"[{' '.join(res)}]"

  def add(self, n):
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      cur[v] = None
      cur = cur[v]
  
  def remove(self, n) : #O(self.depth)
    cur = self.root
    path = [cur] #자식 노드가 없어진 노드들을 확인하기 위해 경로 저장
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False #존재하지 않는 노드를 삭제하려고 할 때
      cur = cur[v]
      path.append(cur)

    #TODO: implement backtrack to remove unused nodes

    return True

  def query(self, n): # O(self.depth), trie 상에 존재하는 원소중에 n과 XOR 연산했을 때의 최솟값
    cur = self.root
    res = 0
    for i in reversed(range(self.depth)) :
      v = not (n >> i) & 1
      if v in cur :
        cur = cur[v]
      else :
        res += 1 << i
        cur = cur[not v]
    return res

def sol() :
  N, M = map(int, input().split())
  L = set(map(int, input().split()))
  Q = []
  for _ in range(M) :
    Q.append(int(input()))
  trie = BinaryTrie(24)

  MAX, MIN = -1, 300000 + 7777
  for v in L: 
    trie.add(v)
    MAX = max(MAX, v)
    MIN = min(MIN, v)
    
  offset = MAX - MIN + 1 #구간 [MIN, MAX] 사이에 존재하지 않는 값 중 가장 작은 값이 최솟값으로 부터 얼마나 떨어져 있는지의 offset, 즉 MIN + offset 이 그러한 최솟값이다.
  for i in range(0, MAX-MIN+1) :
    if MIN + i not in L :
      offset = i
      break

  # naive
  L2 = L.copy()
  for q in Q :
    tmp = []
    for v in L2 :
      tmp.append(q ^ v)
    L2 = tmp
    print(tmp)

  delta = []
  x = 0
  for q in Q :
    x ^= q
    res = trie.query(x)
    delta.append(res - MAX) #원본 수열로부터 얼만큼 움직였는지 저장

  ans = []
  for d in delta: 
    assert MIN + d >= 0
    if d > 0 : #문제의 정의에 의해 수열의 어떤 값도 0보다 항상 크다. 수열이 기존 수열보다 오른쪽으로 이동했다면, 수열의 최솟값이 0보다 오른쪽에 있다는 뜻이다.
      ans.append(0)
    elif MIN + d > 0 : #수열이 왼쪽으로 했거나, 그대로 있었지만 그 최솟값이 0보다 크다면 mex(L)은 0이다.
      ans.append(0)
    else: #문제에 정의에 의해 이 경우 MIN + d == 0, offset이 정답이다.
      ans.append(offset) 

  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

- 수열이 xor했을 때 그대로 이동한다는 관찰이 있었지만, 그 수열에 0이 존재하면 반례가 생겨서 실패한 풀이이다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class Node:
  def __init__(self):
    self.child = [None, None]
  
  def __setitem__(self, k: bool, _):
    if self.child[k] is None: self.child[k] = Node() 
  
  def __getitem__(self, k: bool):
    if self.child[k]: return self.child[k]
    self.child[k] = Node()
    return self.child[k]
  
  def __contains__(self, k: bool):
    return self.child[k] is not None

class BinaryTrie:
  def __init__(self, depth=32):
    self.root = Node()
    self.depth = depth

  def __contains__(self, n) :
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False
      cur = cur[v]
    return True
  
  def __repr__(self): #print 모든 leaf노드의 값 출력. O(2^N) 이지만 보통 sparse하므로 보통 O(|leaf nodes|)
    cur = self.root
    if all(cur.child) : return "[]"
    S = [(cur, 0, 0)]
    res = []
    while S :
      u, d, v = S.pop()
      if not u.child[0] and not u.child[1] and d == self.depth:
        res.append(str(v))
        continue
        
      for i in range(2) :
        if u.child[i] :
          S.append((u.child[i], d+1, v*2+i))
      
    return f"[{' '.join(res)}]"

  def add(self, n):
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      cur[v] = None
      cur = cur[v]
  
  def remove(self, n) : #O(self.depth)
    cur = self.root
    path = [cur] #자식 노드가 없어진 노드들을 확인하기 위해 경로 저장
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False #존재하지 않는 노드를 삭제하려고 할 때
      cur = cur[v]
      path.append(cur)

    #TODO: implement backtrack to remove unused nodes

    return True

  def query(self, n): # O(self.depth), trie 상에 존재하는 원소중에 n과 XOR 연산했을 때의 최솟값
    cur = self.root
    res = 0
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if v in cur : #prefer same bit
        cur = cur[v]
      else :
        res += 1 << i
        cur = cur[not v]
    return res

class ST:
  def __init__(self, L, f, default=0):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __getitem__(self, i):
    return self.L[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.L[i] = v
    i //= 2
    while i:
      self.L[i] = self._f(self.L[2 * i], self.L[2 * i + 1])
      i //= 2

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s //= 2
      e //= 2

    return self._f(l, r)

MAX = 300000
def sol() :
  N, M = map(int, input().split())
  L = set(map(int, input().split()))
  Q = []
  for _ in range(M) :
    Q.append(int(input()))
  trie = BinaryTrie(24)

  L2 = [*reversed(range(MAX+1))]
  for v in L: 
    trie.add(v)
    L2[v] = -1

  st = ST(L2, max, default = -1)

  # naive
  # L2 = L.copy()
  # for q in Q :
  #   tmp = []
  #   for v in L2 :
  #     tmp.append(q ^ v)
  #   L2 = tmp
  #   print(tmp)

  ans = []
  x = 0
  for q in Q :
    x ^= q
    match x :
      case _ if trie.query(x) : # 수열의 최솟값이 0보다 크므로 mex(L)은 0이다.
        ans.append(0)
      case _:
        debug(st.L[1])
        ans.append((MAX - st.query(0, N+1)) ^ x)
    print(ans[-1])

  # sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

;; ;; 0 다음에 연속하는 숫자가 없다면
;; 2 1
;; 0 2
;; 1
;; ;; 정답: 1
;; 4 1
;; 1 2 3 4
;; 1
;; ;; 정답: 3
;; 4 1
;; 0 1 3 4
;; 1
;; ;; XOR를 해서 mex가 바뀌는 경우
;; 3 1
;; 0 2 3
;; 3
;; 예제 입력 1
2 2
1 3
1
3
;;
4 3
0 1 5 6
1
2
4
;;
5 4
0 1 5 6 7
1
1
4
5